Credit: Đoàn Vĩnh Nhân - Toán Tin K68 - 20237376

### Phương pháp Gauss - Jordan
+ Input: Ma trận $A_{m\times n},B_{m\times q}$
+ Output: ma trận $X_{n\times (q+k)}$, với $k$ là số hàng bằng 0 sau khi khử $AB$. Các phần tử trên cột q+1, q+2,...q+k đặc trưng cho sự phụ thuộc của các phần tử còn lại của $X$ vào các phần tử tự do.

Ý tưởng thuật toán: khử ma trận tương tự gauss nhưng khác biệt ở cách chọn phần tử khử. Hạn chế sai số trong quá trình khử do chia cho số gần bằng 0

1. Xét ma trận mở rộng A|B, chọn phần tử khử a trong ma trận
   + Ưu tiên 1: $|a|=1$
   + Ưu tiên 2: $|a| = \max\{|a_{ij}|\}$
   + Chọn phần tử khử thứ k có hàng và cột khác với k-1 phần tử trước đó

2. Cho $t$ chạy từ $1$ đến $m,\, t \ne p$ thực hiện: $$ L_t- \dfrac{a_{tq}}{a_{pq}}L_p\Rightarrow L_t \quad (*)$$


Thuật toán

1. Tạo 2 danh sách rowuse, coluse rỗng, dùng lưu hàng, cột của các phần tử khử đã chọn.
2. Tìm kiếm phần tử khử theo ưu tiên (1), (2) ở trong những hàng, cột khác các phần tử trong rowuse, coluse
3. Khử ma trận A|B theo công thức (*). Lưu vị trí cột, hàng của phần tử khử vào rowused,colused.
4. Nếu rowuse == row hoặc coluse == col hoặc pivot == 0 (trường hợp có hàng bằng 0) đến bước 5, nếu không quay lại bước 2
5. Chuẩn hoá A|B : chia từng hàng cho pivot (nếu có) và sắp xếp về dạng bậc thang

6. Trả về kết quả nghiệm (vô nghiệm, nghiệm duy nhất, nghiệm tổng quát của trường hợp vô số nghiệm)
+ Chuyển các cột không có phần tử khử của $A$ ra sau $B$ (Các phần tử tự do)
+ Loại bỏ các hàng bằng 0
+ Trả về B (giờ đây B chính là X do các ma trận A giờ đây đã là ma trận đơn vị)

In [1]:
from sympy import Matrix, zeros, N
import numpy as np

#Hàm này được thêm vào để loại bỏ các giá trị nhỏ hơn 10⁻¹⁰, đảm bảo chúng được coi là 0.
def chop_small_values(mat):
    if isinstance(mat, Matrix):
        return mat.applyfunc(lambda x: 0 if abs(x) < 1e-10 else x)
    else:
        return 0 if abs(mat) < 1e-10 else mat

#Hàm tìm kiếm phần tử khử, trả về giá trị và vị trí của phần tử khử
def findpivot(AB,rowA,colA,rowused,colused):
    #Hàng và cột chưa dùng
    row = list(set(range(rowA)) - set(rowused))
    col = list(set(range(colA)) - set(colused))
    
    #Ưu tiên 1 và ưu tiên 2
    firts_prioritize_element = [AB[i, j] for i in row for j in col
                                if np.abs(float(AB[i, j])) == float(int(1))]
    if firts_prioritize_element:
        pivot = min(firts_prioritize_element, key=abs)
    else:
        second_prioritize_element = [AB[i, j] for i in row for j in col]
        pivot = max(second_prioritize_element, key=abs)
    positions = [[i, j] for i in row for j in col if AB[i, j] == pivot]
    return pivot, positions
#Hàm khử gauss_jordan
def gauss_jordan(A,B):

    A, B = Matrix(N(A)), Matrix(N(B))
    rowused = []
    colused = []
    rowA, colA = A.shape
    AB = chop_small_values(A.row_join(B))
    display(AB)
    standardization = []

    #Tìm pivot và khử theo công thức (1)
    while ((len(rowused)!=rowA) and (len(colused)!=colA)):
        pivot = findpivot(AB,rowA,colA,rowused,colused)[0]
        if (abs(pivot) < 1e-10): break
        pivotrow, pivotcol = findpivot(AB,rowA,colA,rowused,colused)[1][0]
        for i in [x for x in range(rowA) if x!=pivotrow]:
            AB[i,:] = AB[i,:] - AB[pivotrow,:]*AB[i,pivotcol]/pivot
        rowused.append(pivotrow)
        colused.append(pivotcol)
        standardization.append([pivotrow,pivotcol])
        print("Khử")
        AB = chop_small_values(AB)
        display(AB)

    #Chuẩn hoá, chia các hàng cho pivot
    print("Chuẩn hoá")
    for i in range(len(standardization)):
        pivotrow, pivotcol = standardization[i]
        AB[pivotrow,:] = AB[pivotrow,:]/AB[pivotrow,pivotcol]
    return chop_small_values(AB), colused
#Danh sách các vị trí khác 0 đầu tiên của từng hàng, dùng để sắp xếp AB theo cấu trúc bậc thang
def indexpivot(AB):
    rowAB, colAB = AB.shape
    index = []
    for row in range(rowAB):
        for col in range(colAB):
            if abs(AB[row, col]) > 1e-10: 
                index.append(col)
                break
            if col == colAB - 1:
                index.append(col + 1)
    return index
#Giải AX=B
def gauss_jordan_solve(A, B):
    #Ma trận đã khử AB
    A, B = Matrix(N(A)), Matrix(N(B))
    AB,colused = gauss_jordan(A, B)
    display(AB)
    #Kiểm tra trường hợp vô nghiệm
    for i in indexpivot(AB):
        if i >= A.shape[1] and i < AB.shape[1]:
            print("Hệ vô nghiệm")
            return 
    
    print(f"Số phần tử tự do là {len(list(set(range(A.shape[1]))-set(colused)))}: X{sorted([x+1 for x in list(set(range(A.shape[1]))-set(colused))])}.")

    return

In [2]:
A = Matrix(np.loadtxt("GJ_input_A.txt"))
print("A =")
display(A)
print("B =")
B = Matrix(np.loadtxt("GJ_input_B.txt"))
display(B)
print("A|B=")
gauss_jordan_solve(A,B)

A =


Matrix([
[-3.0,  5.0,  5.0, -11.0,  0.0],
[-2.0, 11.0, 17.0, -35.0,  2.0],
[-4.0, -9.0,  2.0,   6.0, 12.0],
[ 9.0, -3.0, 25.0, -39.0, 16.0],
[ 6.0,  4.0, -1.0,  -7.0, -6.0]])

B =


Matrix([
[-4.0, -1.0],
[-7.0,  6.0],
[ 7.0, -8.0],
[ 8.0, 19.0],
[-2.0, 10.0]])

A|B=


...

Khử


Matrix([
[                0,    0,    0,      0,                0,                 0,                 0],
[-10.3333333333334,    0,    0,      0, 6.38787878787879, -1.69090909090909, -13.7151515151515],
[                0, -1.0,    0,      0, 1.35757575757576, 0.915151515151515, -0.16969696969697],
[                0,    0,    0, -990.0, 443.999999999998,  96.0000000000009,  192.000000000002],
[                0,    0, -1.0,      0,                0,                 0,                 0]])

Chuẩn hoá


Matrix([
[  0,   0,   0,   0,                  0,                   0,                  0],
[1.0,   0,   0,   0, -0.618181818181816,   0.163636363636363,   1.32727272727273],
[  0, 1.0,   0,   0,  -1.35757575757576,  -0.915151515151515,   0.16969696969697],
[  0,   0,   0, 1.0, -0.448484848484847, -0.0969696969696978, -0.193939393939396],
[  0,   0, 1.0,   0,                  0,                   0,                  0]])

Số phần tử tự do là 1: X[5].
